🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Author : Nicolas Grosjean*

*Date : 02/11/2025*

**Description :**

This Jupyter Notebook analyses the C2C data from parquet files

In [12]:
import folium
import pandas as pd

In [2]:
import os

os.chdir("../..")
print("Working directory set to the root of the project")

Working directory set to the root of the project


In [3]:
from src.processors.c2c import C2CBusStopsProcessor

In [4]:
stops_gdf = C2CBusStopsProcessor.fetch(reload_pipeline=False)
stops_gdf.head()

,gtfs_id,navitia_id,osm_id,name,description,line_gtfs_ids,line_osm_ids,network,network_gtfs_id,geometry,other
0,None,stop_area:OGE:GEN15846,None,"Le Haut-Bréda, Pinsot le Village (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (679047.781 5677868.337),"{'line': 'Bus 79 - ALLEVARD ECOLE PLEIADE', 's..."
1,None,stop_area:OGE:GEN15852,None,"Le Haut-Bréda, Hot Pic Belle Etoile (Le Haut-B...",None,[],[],Mobilités M - TouGo,None,POINT (678910.858 5678497.283),"{'line': 'Bus 79 - ALLEVARD ECOLE PLEIADE', 's..."
2,None,stop_area:OGE:GEN15850,None,"Le Haut-Bréda, Chinfert (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (678788.406 5679655.482),"{'line': 'Bus 79 - ALLEVARD ECOLE PLEIADE', 's..."
3,None,stop_area:OGE:GEN15080,None,"Le Haut-Bréda, la Piat (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (678378.751 5675153.444),"{'line': 'Bus 79 - ALLEVARD ECOLE PLEIADE', 's..."
4,None,stop_area:OGE:GEN13054,None,"Domène, Domène Mairie (Domène)",None,[],[],Mobilités M - Tag,None,POINT (649900.998 5653440.123),{'line': 'Bus 15 - GRENOBLE Verdun - Préfectur...


In [7]:
expanded = stops_gdf["other"].apply(pd.Series)
expanded_stops_gdf = pd.concat([stops_gdf.drop(columns=["other"]), expanded], axis=1)
expanded_stops_gdf.head()

,gtfs_id,navitia_id,osm_id,name,description,line_gtfs_ids,line_osm_ids,network,network_gtfs_id,geometry,line,srid,stoparea_id
0,None,stop_area:OGE:GEN15846,None,"Le Haut-Bréda, Pinsot le Village (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (679047.781 5677868.337),Bus 79 - ALLEVARD ECOLE PLEIADE,3857,1
1,None,stop_area:OGE:GEN15852,None,"Le Haut-Bréda, Hot Pic Belle Etoile (Le Haut-B...",None,[],[],Mobilités M - TouGo,None,POINT (678910.858 5678497.283),Bus 79 - ALLEVARD ECOLE PLEIADE,3857,2
2,None,stop_area:OGE:GEN15850,None,"Le Haut-Bréda, Chinfert (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (678788.406 5679655.482),Bus 79 - ALLEVARD ECOLE PLEIADE,3857,3
3,None,stop_area:OGE:GEN15080,None,"Le Haut-Bréda, la Piat (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (678378.751 5675153.444),Bus 79 - ALLEVARD ECOLE PLEIADE,3857,4
4,None,stop_area:OGE:GEN13054,None,"Domène, Domène Mairie (Domène)",None,[],[],Mobilités M - Tag,None,POINT (649900.998 5653440.123),Bus 15 - GRENOBLE Verdun - Préfecture / DOMÈNE...,3857,5


In [15]:
expanded_stops_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1587 entries, 0 to 1586
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   gtfs_id          0 non-null      object  
 1   navitia_id       1587 non-null   object  
 2   osm_id           0 non-null      object  
 3   name             1587 non-null   object  
 4   description      0 non-null      object  
 5   line_gtfs_ids    1587 non-null   object  
 6   line_osm_ids     1587 non-null   object  
 7   network          1587 non-null   object  
 8   network_gtfs_id  0 non-null      object  
 9   geometry         1587 non-null   geometry
 10  line             1587 non-null   object  
 11  srid             1587 non-null   int64   
 12  stoparea_id      1587 non-null   int64   
dtypes: geometry(1), int64(2), object(10)
memory usage: 161.3+ KB


In [20]:
# Add a base OSM map centered on Grenoble
m = folium.Map(location=[45.1885, 5.7245], zoom_start=12, tiles="OpenStreetMap")

# Add Waymarked Trails hiking layers
folium.TileLayer(tiles="WaymarkedTrails.hiking").add_to(m)

folium.GeoJson(
    expanded_stops_gdf[["navitia_id", "name", "line", "geometry"]],
    name="Stop areas",
    zoom_on_click=True,
    marker=folium.Marker(icon=folium.Icon(icon="star")),
    tooltip=folium.GeoJsonTooltip(fields=["name", "line"]),
    popup=folium.GeoJsonPopup(fields=["navitia_id", "name", "line"]),
).add_to(m)
m